In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 30)

c:\Users\totog\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Chargement des données

On commence par charger le fichier qui indique les réponses des participants au questionnaire. Les réponses aux questions sont codées par des nombres, la signification des codes est fournie dans un fichier annexe de documentation. On va donc devoir jongler entre les deux.

In [2]:
df_brut = pd.read_excel("Data/Etude sur les Français et l_information/les-francais-et-l-information-arcom-2024-base-anonymisee.xlsx")
df_doc = pd.read_excel("Data/Etude sur les Français et l_information/Documentation/les-francais-et-l-information-arcom-2024-datamap.xlsx", sheet_name='TEXTS')

print("format des données brutes: ", df_brut.shape)

format des données brutes:  (3346, 1076)


## Premier nettoyage des données

Le fichier des données brutes contient des colonnes récapitulatives qui donnent une information redondante, on va supprimer celles-ci pour alléger le dataset.

In [3]:
# Suppression des colonnes "récap" dans la documentation
colonnes_recap = list(df_doc[df_doc["FR:L"].str.contains("Récap", case=False, na=False)]["NAME"])
colonnes_recap += list(df_doc[df_doc["FR:L"].str.contains("Recap", case=False, na=False)]["NAME"])
colonnes_recap += ["INT3_R7", "RSINFO_2_LR_R2", "RSINFO_2_LR_R3", "RSINFO_2_LR_R4", "COMPLOT2_R2"]

# Suppression des colonnes "récap" dans les données
colonnes_recap = [col for col in df_brut.columns if any(mot in col for mot in colonnes_recap)]
df_new = df_brut.drop(columns=colonnes_recap)

print(f"{len(colonnes_recap)} colonnes ont été supprimées. Il en reste {df_new.shape[1]}.")

537 colonnes ont été supprimées. Il en reste 539.


On va maintenant parcourir le fichier de documentation et construire un dictionnaire qui nous donne, pour chaque question sa documentation spécifique afin de faciliter les prochains programmes.

Le fichier est une table dans laquelle certaines lignes indiquent l'intitulé d'une question (la colonne "TYPE" indique alors "TITLE") et les lignes suivantes donnent alors le code correspondant aux différentes réponses (la colonne "TYPE" indique alors "LABEL").

On parcours donc le fichier et dès qu'on rencontre une ligne TITLE, on récupère toutes les lignes LABEL qui suivent jusqu'au prochain TITLE.

On obtient un dataframe pour chaque question mais ce n'est pas suffisant. En effet pour certaines questions, la documentation des codes est indiquée dans une autre question qui la précède. De plus pour certaines question, seule une partie du dataframe nous interesse car le reste indique la signification des colonnes que nous traiterons à part. Malheureusement, on ne peut que modifier ceci à la main.

In [4]:
# Dictionnaire pour stocker les résultats
documentation = {}

# Parcours du DataFrame
current_key = None  # Clé pour le dictionnaire, c'est l'indentificateur de la question
start_index = None  # Indique l'index de début des lignes LABEL qui correspondent au réponses à une qu

for i, row in df_doc.iterrows():
    if row["TYPE"] == "TITLE":
        # Sauvegarde le sous-DataFrame précédent si un TITLE est déjà en cours
        if current_key is not None and start_index is not None:
            documentation[current_key] = df_doc.iloc[start_index:i]

        # Met à jour la clef courant et le début des lignes LABEL
        current_key = row["NAME"]
        start_index = i + 1

# Ajoute la documentation de la dernière question après le dernier TITLE
if current_key is not None and start_index is not None:
    documentation[current_key] = df_doc.iloc[start_index:]
    
# Supprime les entrées qui correspondent à des colonne récap
keys_to_remove = {"_".join(elem.split("_")[:2]) for elem in colonnes_recap}  
documentation = {key: value for key, value in documentation.items() if key not in keys_to_remove}

# Supprime quelques entrées inutiles
del documentation["NOU1_R"], documentation["NBR_NEWS1SOURCES_R1"], documentation["NBR_NEWS1SOURCES_R3"]
del documentation["RSINFO_2_LR_R2"], documentation["RSINFO_2_LR_R3"], documentation["RSINFO_2_LR_R4"],
del documentation["SOURCES1A_DOMR_R"], documentation["LEVEL_A"], documentation["RECORD"], documentation["UUID"]

noms_colonnes = {}

# Reprécise la documentation pour certaines questions
documentation["RS16_R"] = documentation["RS16_R"].loc[132:134]
documentation["INT2_R"] = documentation["INT1_R"]
documentation["MOTIV_R"] = documentation["MOTIV_R"].loc[216:219]
documentation["NEWS1_R"] = documentation["NEWS1_R"].loc[252:257]
documentation["NEWS1SOURCES_LR7_R"] = documentation["NEWS1SOURCES_LR6_R"]
documentation["NEWS1SOURCES_LR8_R"] = documentation["NEWS1SOURCES_LR6_R"]
documentation["NEWS1SOURCES_LR9_R"] = documentation["NEWS1SOURCES_LR6_R"]
documentation["DEVICEXMOMENTS_R1"] = documentation["DEVICEXMOMENTS_R1"].loc[338:345]
documentation["FORMAT3_LR2_FORMAT3_2_R"] = documentation["FORMAT3_LR1_FORMAT3_1_R"]
documentation["FORMAT3_LR3_FORMAT3_3_R"] = documentation["FORMAT3_LR1_FORMAT3_1_R"]
documentation["FORMAT2_LR3_FORMAT2_3_R"] = documentation["FORMAT2_LR1_FORMAT2_1_R"]
documentation["PAY_R"] = documentation["PAY_R"].loc[369:370]
documentation["AGREG_R"] = documentation["AGREG_R"].loc[388:390]
documentation["NOTIF_R"] = documentation["PAY_R"].loc[369:370]
documentation["RSINFO_2_LR_R"] = documentation["PAY_R"].loc[369:370]
documentation["ENGAR_R"] = documentation["ENGAR_R"].loc[473:475]
documentation["INF1_R"] = documentation["INF1_R"].loc[633:638]
documentation["INF3_R"] = documentation["INF3_R"].loc[691:694]
documentation["ATTSOURCES_R"] = documentation["ATTSOURCES_R"].loc[716:719]
documentation["OP2_R"] = documentation["OP2_R"].loc[757:760]
documentation["COMPLOT1_R"] = documentation["PAY_R"].loc[369:370]
documentation["COMPLOT2_R"] = documentation["PAY_R"].loc[369:370]
documentation["VALEURJ_R"] = documentation["INF1_R"].loc[633:638]
documentation["ATTENTESJ_R"] = documentation["ATTENTESJ_R"].loc[802:804]
documentation["DEFIANCEJ_R"] = documentation["DEFIANCEJ_R"].loc[834:837]
documentation["CONNAISSALGOA_R"] = documentation["CONNAISSALGOA_R"].loc[862:863]
documentation["OPTI_R"] = documentation["OPTI_R"].loc[904:907]
documentation["ATTA_R"] = documentation["ATTA_R"].loc[924:927]
documentation["DEM_R"] = documentation["DEM_R"].loc[948:951]
documentation["CONFINST_R"] = documentation["INF3_R"].loc[691:694]
documentation["PERS_R"] = documentation["PAY_R"].loc[369:370]
documentation["VA1_R"] = documentation["VA1_R"].loc[1009:1013]


# Affichage de la documentation
for key, sub_df in documentation.items():
    print(f"Bloc '{key}':")
    print(sub_df)
    print()


Bloc 'CIBLE':
  HEADER TEXT   NAME   TYPE  CODE  VALUE  FORMULA  BASE  GROUPID  \
4        TEXT  CIBLE  LABEL   1.0    NaN      NaN   NaN      NaN   
5        TEXT  CIBLE  LABEL   2.0    NaN      NaN   NaN      NaN   
6        TEXT  CIBLE  LABEL   3.0    NaN      NaN   NaN      NaN   

                    FR:L  
4  France métropolitaine  
5                DOM TOM  
6                DOM - 2  

Bloc 'SOURCE_BASE_R1':
   HEADER TEXT            NAME   TYPE  CODE  VALUE  FORMULA  BASE  GROUPID  \
8         TEXT  SOURCE_BASE_R1  LABEL   1.0    NaN      NaN  -1.0      1.0   
9         TEXT  SOURCE_BASE_R1  LABEL   2.0    NaN      NaN   NaN      1.0   
10        TEXT  SOURCE_BASE_R1  LABEL   3.0    NaN      NaN   NaN      1.0   

              FR:L  
8             CATI  
9   CAWI Métropole  
10  CAWI Outre-mer  

Bloc 'POIDS03':
Empty DataFrame
Columns: [HEADER TEXT, NAME, TYPE, CODE, VALUE, FORMULA, BASE, GROUPID, FR:L]
Index: []

Bloc 'INTER':
Empty DataFrame
Columns: [HEADER TEXT, NAME, TYP

On va maintenant utiliser ce dictionnaire de documentations pour transformer toutes les valeurs codées numériquement par la réponse qui correspond dans le DataFrame. Celà permettra de l'utiliser plus simplement via un outil de création de dashboard par exemple.

In [5]:
# La colonne "DEVICEXMOMENTS_R1" qui doit être traitée avant "DEVICEXMOMENTS_R"
liste_col = [col for col in df_new.columns if col.startswith("DEVICEXMOMENTS_R1")]
doc = documentation["DEVICEXMOMENTS_R1"]
for col in liste_col:
    df_new[col] = df_new[col].apply(lambda x: doc[doc["CODE"]==x]["FR:L"].iloc[0] if x in doc["CODE"].unique() else x)

# Parcours de la doc 
for i, question in enumerate(documentation.keys()):
    
    # Colonnes concernées par cette question et documentation associée
    liste_col = [col for col in df_new.columns if col.startswith(question)]
    doc = documentation[question]
    
    # Remplacement des codes par leur signification
    for col in liste_col:
        df_new[col] = df_new[col].apply(lambda x: doc[doc["CODE"]==x]["FR:L"].iloc[0] if x in doc["CODE"].unique() else x)
    
    # Checking de l'avancement 
    print(f"\rAvancement: étape {i:03d} / {len(documentation.keys()):03d}", end='', flush=True)

# Colonne NOU1_R
df_new["NOU1_R"] = df_new["NOU1_R"] -1

# Affichage du résultat
df_new


Avancement: étape 002 / 094

C:\Users\totog\AppData\Local\Temp\ipykernel_18736\3501160386.py:16: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  df_new[col] = df_new[col].apply(lambda x: doc[doc["CODE"]==x]["FR:L"].iloc[0] if x in doc["CODE"].unique() else x)


Avancement: étape 034 / 094

KeyboardInterrupt: 

Les réponses aux questions sont maintenant lisibles mais ce n'est pas fini, il reste à indiquer à quoi correspondent les noms de colonnes. On pourrait se dire qu'il suffit de remplacer ces noms par la question correspondante mais c'est un peu plus compliqué. Il existe 3 types de questions dans ce questionnaire

- Question simple à choix simple (Par exemple "Indiquez votre tranche d'âge"): Dans ce cas, une seule colonne correspond à la question et la réponse est donnée systématiquement.
- Question multiple à choix simple (Par exemple "Voici 5 affirmations, indiquez si vous êtes d'accord, pas d'accord,..."): Dans ce cas plusieurs colonnes correspondent à la même question et la réponse est donnée systématiquement.
- Question simple à choix multiple (Par exemple "Parmi ces 8 chaines de TV, lesquelles regardez-vous?"): Dans ce cas il y a autant de colonnes que de réponses possibles. La première colonne indique la première réponse donnée par l'interrogé, etc,.. Si l'interrogé a donné 3 réponses, les colonnes 1 à 3 les contiennent et les suivantes sont vides (NaN).

De plus, les questions pouvant être très longues, je préfère fournir dans un dictionnaire à part les énoncés des questions et les colonnes correspondantes, de manière à garder des noms de colonne simple et facile à utiliser comme "RS16_R_1".

In [76]:
def multiple_choice(question, nb_reponses):
    for i in range(nb_reponses):
        questionnaire[question + f"_{i+1}" ] = f"Réponse {i+1}" 

questionnaire = {
    "LEVEL_A": "Records",
    "RECORDS": "Numéro d'enregistrement",
    "UUID": "Identifiant du partcipant",
    "CIBLE": "Cible",
    "SOURCE_BASE_R1": "Source",
    "POIDS03": "Poids",
    "RS1_R": "Genre",
    "RS2C_RECODE_AG_R": "Age",
    "RS3_R":"Situation professionnelle",
    "RS4_R": "Profession",
    "CSPIND_R": "CSP",
    "RS5_R": "Niveau d'étude",
    "UDA9_R": "Région en 9",
    "UDA5_R": "Région en 5",
    "DPT_DOM": "DOM",
    "CC_R": "Taille agglo",
    "RS7_R":"Nombre de personnes dans le foyer",
    "RS14_R": "Revenu mensuel net du foyer",
    "ILLECTRO_R": "Dernière utilisation d'internet",
    "RS15_R": "Fréquence d'utilisation d'internet",
    "FREQ_INTERNET_R": "Fréquence d'utilisation d'internet (recode)",
    
    "RS16_R": "Équipement média",
    "RS16_R_1": "un poste de TV",
    "RS16_R_2": "un poste de radio, une chaine hifi",
    "RS16_R_3": "un autoradio",
    "RS16_R_4": "un smartphone",
    "RS16_R_5": "un ordinateur fixe",
    "RS16_R_6": "un ordianteur portable",
    "RS16_R_7": "une tablette tactile",
    "RS16_R_8": "la google chromecast, le firestick amazon ou une box appleTV",
    "RS16_R_9": "une enceinte connectée avec assistant vocal",
    "RS16_R_10": "un casque VR",
    "RS16_R_11": "une montre connectée",
    "RS16_R_12": "une console de jeux",
    
    "INT1_R": "Intérêt global pour l'information",
    
    "INT2_R": "Intérêt détaillé pour l'information sur un thème",
    "INT2_R_1": "Actualité internationale",
    "INT2_R_2": "Actualité nationale",
    "INT2_R_3": "Actualité locale / régionale ",
    "INT2_R_4": "Politique",
    "INT2_R_5": "Économie",
    "INT2_R_6": "Société (justice, emplois, fait-divers)",
    "INT2_R_7": "Environnement",
    "INT2_R_8": "Santé, mode de vie et bien-être",
    "INT2_R_9": "Culture et divertissement",
    "INT2_R_10": "Sport",
    "INT2_R_11": "People",
    "INT2_R_12": "Science et technologie",
    
    "VERSION_QUESTIO_R": "Version du questionnaire",
    
    "INT3_R": "Sentiment d'être bien informé sur les thèmes d'intérêt",
    "INT3_R": "Intérêt détaillé pour l'information sur un thème",
    "INT3_R_1": "Actualité internationale",
    "INT3_R_2": "Actualité nationale",
    "INT3_R_3": "Actualité locale / régionale ",
    "INT3_R_4": "Politique",
    "INT3_R_5": "Économie",
    "INT3_R_6": "Société (justice, emplois, fait-divers)",
    "INT3_R_7": "Environnement",
    "INT3_R_8": "Santé, mode de vie et bien-être",
    "INT3_R_9": "Culture et divertissement",
    "INT3_R_10": "Sport",
    "INT3_R_11": "People",
    "INT3_R_12": "Science et technologie",
    "MOTIV_R": "Motivations pour s'informer",
        
    "NEWS1_R": "Fréquence d'utilisation des modes d'information",
    "NEWS1_R_1": "En écoutant la radio ou des podcast",
    "NEWS1_R_1": "En regardant la TV (replay ou direct)",
    "NEWS1_R_1": "En lisant un journal ou magazine",
    "NEWS1_R_1": "En vous connectant directement sur un site internet",
    "NEWS1_R_1": "En utilisant un moteur de recherche",
    "NEWS1_R_1": "En utilisant un portail d'actualité",
    "NEWS1_R_1": "En consultant les réseaux sociaux",
    "NEWS1_R_1": "En consultant les plateformes vidéo en ligne",
    "NEWS1_1B_R": "Support d'écoute radio ou podcast",
    "NEWS1_2B_R": "Support pour regarder la TV",
    "NEWS1_3B_R": "Support de lecture de journeaux ou magazines",
    "NEWS1SOURCES_LR6_R": "Source des informations consultées via un moteur de recherche",
    "NEWS1SOURCES_LR7_R": "Source des informations consultées via un portail d'actualité",
    "NEWS1SOURCES_LR8_R": "Source des informations consultées via les réseaux sociaux",
    "NEWS1SOURCES_LR8_R": "Source des informations consultées via des vidéos en lignes",
    "NEWS1BIS_R": "Mode principal d'information",
    
    "DEVICEXMOMENTS_R": "Moment pour s'informer",
    "DEVICEXMOMENTS_R1": "Moment pour s'informer par équipement",
    "DEVICEXMOMENTS_R1_1": "Le matin avant 9h",
    "DEVICEXMOMENTS_R1_2":"Entre 9h et 12h",
    "DEVICEXMOMENTS_R1_3":"Entre 12h et 14h",
    "DEVICEXMOMENTS_R1_4":"Entre 14h et 18h",
    "DEVICEXMOMENTS_R1_5":"Entre 18h et 20h",
    "DEVICEXMOMENTS_R1_6":"Entre 20h et 22h",
    "DEVICEXMOMENTS_R1_7":"Après 22h",
    
    "FORMAT_R": "Formats privilégiés pour s'informer",
    "FORMAT3_LR1_FORMAT3_1_R": "L'audio, le son : Longueur préférée",
    "FORMAT2_LR1_FORMAT2_1_R": "L'audio, le son : En direct ou en replay",
    "FORMAT3_LR2_FORMAT3_2_R": "L'écrit : Longueur préférée",
    "FORMAT3_LR3_FORMAT3_3_R": "La vidéo : Longueur préférée",
    "FORMAT2_LR3_FORMAT2_3_R": " La vidéo : En direct ou en replay",
    
    "PAY_R": "Paye pour s'informer",
    "PAY_R_1": "J'ai acheté des journaux ou magazines à l'unité en point de vente presse",
    "PAY_R_2": "J'ai acheté des articles de journaux ou magazines à l'unité en ligne",
    "PAY_R_3": "J'ai payé un abonnement à un ou plusieurs journaux ou magazines",
    "PAY_R_4": "J'ai payé un abonnement à un kiosque numérique",
    "PAY_R_5": "J'ai payé un abonnement à un site d'information en ligne",
    "PAY_R_6": "J'ai payé un abonnement à une newsletter",
    "PAY_R_7": "J'ai payé pour d'autres abonnements (Twitch, …)",

    "AGREG_R": "Agrégateur d'actualité consultés",
    "AGREG_R_1": "Flipboard",
    "AGREG_R_2": "Upday",
    "AGREG_R_3": "Google actualités",
    "AGREG_R_4": "Microsoft start",
    "AGREG_R_5": "Opéra news",
    "AGREG_R_6": "Orange actu",
    "AGREG_R_7": "Portail MSN",
    "AGREG_R_8": "Yahoo actualités",
    
    "NOTIF_R": "Activation des notification",
    "RSINFO1_LR_R": "Utilisation des réseaux sociaux et plateformes en général",
    "RSINFO_2_LR": "Utilisation des réseaux sociaux et plateformes pour s'informer",
    "RSINFO_2_LR_R_1":"Facebook",
    "RSINFO_2_LR_R_2":"Instagram",
    "RSINFO_2_LR_R_3":"X",
    "RSINFO_2_LR_R_4":"TikTok",
    "RSINFO_2_LR_R_5":"Snapchat",
    "RSINFO_2_LR_R_6":"Linkedin",
    "RSINFO_2_LR_R_7":"WhatsApp",
    "RSINFO_2_LR_R_8":"Telegram",
    "RSINFO_2_LR_R_9":"Youtube",
    "RSINFO_2_LR_R_10":"Reddit",
    "RSINFO_2_LR_R_11":"Discord",
    "RSINFO_2_LR_R_12":"Twitch",
    "RSINFO_2_LR_R_13":"un autre réseau",
    
    "MOTIVRS_R": "Motivations pour s'informer sur les réseaux sociaux",
    "ENGAR_R": "Fréquence des interactions sur les réseaux sociaux",
    "ENGAR_R_1": "Commente, Like, Partage",
    "ENGAR_R_1": "Commenter",
    "ENGAR_R_1":"Partager",
    "ENGAR_R_1": "Liker",
    
    "SOURCES1AR_B1_R": "Marques médias consommées / consultées régulièrement",
    "SOURCES1A_DOMR_R": "Média outre-mer : Marques médias consommées / consultées régulièrement",
    "SOURCES1CR_B3_R": "Magazines, revues : Marques médias consommées / consultées régulièrement",
    "SOURCES1DR_B4_R": "Chaînes TV 1 à 12: Marques médias consommées / consultées régulièrement",
    "SOURCES1ER_B4_R": "Chaînes TV 13 à 99: Marques médias consommées / consultées régulièrement",
    "SOURCES1FR_R": "Réseaux sociaux / Plateformes : Marques médias consommées / consultées régulièrement",
    "TV2AR_R": "Journaux TV regardés régulièrement ",
    "TV2BR_R": "Emissions quotidiennes regardées régulièrement",
    "TV2CR_R": "Magazines d'informations hebdomadaires ou mensuels regardés régulièrement",
    
    "INF1_R": "Curseur: de quelle affirmation vous sentez-vous le plus proche?",
    "INF1_R_1": "Je vais chercher l'information de façon volontaire / L'information vient à moi sans que je la recherche",
    "INF1_R_2": "J'aime que les informations soient présentées de façon sérieuse /  J'aime que les informations soient présentées de façon drôle, décalée",
    "INF1_R_3":"J'aime lorsque les informations sont traitées de façon rapide et directe / J'aime lorsque les informations sont approfondies et mettent les choses en perspective",
    "INF1_R_4": "J'aime les informations présentées de façon factuelle, neutre et objective / J'aime lorsque l'information confronte des opinions engagées, des partis pris",
    
    "DYN1_R": "Dynamique d'information",
    "DYN2_R": "Raisons de moins s'informer",
    "DYN3_R": "Raisons de plus s'informer",
    "EVITE1_R": "Evitement volontaire de contact avec l'information",
    "EVITE2_R": "Actions mises en place pour éviter le contatct avec l'information",
    "CONF1_R": "Je fais confiance en la plupart des informations",
    
    "INF3_R": "Niveau de confiance avec les sources d'informations",
    "INF3_R_1": "Journalistes (de la presse écrite, radio, télévision)",
    "INF3_R_2": "Experts et expertes invités dans les médias (économistes, médecins, scientifiques, avocats, militaires …)",
    "INF3_R_3": "Personnalités politiques du gouvernement",
    "INF3_R_4": "Autres personnalités politiques (militants, syndicats, parlementaires, chefs de partis)",
    "INF3_R_5": "Célébrités (chanteurs, acteurs, sportifs, artistes…)",
    "INF3_R_6": "Créateurs, créatrices de contenus (Influenceurs, Youtubeurs, Streameurs, …)",
    "INF3_R_7": "Associations et acteurs de terrain",
    "INF3_R_8": "Anonymes (posts ou commentaires sur Internet par exemple)",
    "INF3_R_9": "Vos proches (familles, amis, collègues)",

    "ATTSOURCES_R": "Niveau de confiance en la fiabilité de l'information",
    "ATTSOURCES_R_1": "On peut très bien s'intéresser à des informations sans chercher à savoir si elles sont vraies ou fausses",
    "ATTSOURCES_R_2": "Il y a rarement une seule vérité",
    "ATTSOURCES_R_3": "Les informations données par des anonymes sont aussi fiables que celles des journalistes",
    "ATTSOURCES_R_4": "Il faut toujours vérifier la source d'une information",
    "ATTSOURCES_R_5": "Les médias les plus connus sont les plus fiables",
    "ATTSOURCES_R_6": "Les commentaires sur les réseaux sociaux sont souvent très pertinents",
    
    "INFOFIABLEDEF_R": "Définition d'une information fiable",
    "INFOFIABLE_R": "Niveau de facilité d'accès à une information fiable",
    "OP2_R": "Ce qui contribue le plus à se faire une opinion sur un sujet",
    "OP2_R_1": "Les informations dans les grands médias (radio, tv, presse)",
    "OP2_R_2": "Les informations sur les réseaux sociaux et les plateformes de vidéo",
    "OP2_R_3": "Les discussions avec des inconnus dans la vie ou sur Internet",
    "OP2_R_4": "Les échanges avec vos proches (famille, amis, collègues, …)",
    
    "COMPLOT1_R": "Avez-vous déja entendu cette affirmation?",
    "COMPLOT1_R_1": "Les Américains n'ont jamais marché sur la lune",
    "COMPLOT1_R_2": "L'épidémie Covid s'est répandue accidentellement depuis la ville de Wuhan en Chine",
    "COMPLOT1_R_3": "Il existe un complot juif à l'échelle mondiale",
    "COMPLOT1_R_4": "C'est la CIA qui a abattu les tours jumelles à New-York le 11 septembre",
    "COMPLOT1_R_5": "Le gouvernement est de mèche avec l'industrie pharmaceutique pour cacher la nocivité des vaccins",
    "COMPLOT1_R_6": "Lady Diana est décédée dans un accident de voiture sous le Pont de l'Alma",
    "COMPLOT1_R_7": "Le réchauffement climatique actuel n'est pas causé par l'homme",
    "COMPLOT1_R_8": "Donald Trump est le véritable vainqueur des dernières élections américaines",
    
    "COMPLOT2_R": "Pensez-vous que cette affirmation est exacte?",
    "COMPLOT2_R_1": "Les Américains n'ont jamais marché sur la lune",
    "COMPLOT2_R_2": "L'épidémie Covid s'est répandue accidentellement depuis la ville de Wuhan en Chine",
    "COMPLOT2_R_3": "Il existe un complot juif à l'échelle mondiale",
    "COMPLOT2_R_4": "C'est la CIA qui a abattu les tours jumelles à New-York le 11 septembre",
    "COMPLOT2_R_5": "Le gouvernement est de mèche avec l'industrie pharmaceutique pour cacher la nocivité des vaccins",
    "COMPLOT2_R_6": "Lady Diana est décédée dans un accident de voiture sous le Pont de l'Alma",
    "COMPLOT2_R_7": "Le réchauffement climatique actuel n'est pas causé par l'homme",
    "COMPLOT2_R_8": "Donald Trump est le véritable vainqueur des dernières élections américaines",

    "VALEURJ_R": "Curseur: de quelle affirmation vous sentez-vous le plus proche?",
    "VALEURJ_R_1": "Je pense que les informations partagées par les citoyens sont aussi importantes que celles des journalistes  / L’information est un métier, celui des journalistes",
    "VALEURJ_R_2": "Une information de qualité peut être révélée par n’importe qui /  Une information de qualité ne peut être produite que par des journalistes",
    "VALEURJ_R_3": "Les journalistes peuvent nuire à la démocratie / Les journalistes sont essentiels au bon fonctionnement de notre démocratie",
    "VALEURJ_R_4": "Les journalistes défendent toujours la pensée dominante / Les journalistes reflètent bien la diversité des opinions",
    "VALEURJ_R_5": "Les journalistes cherchent surtout à faire de l’audience /  Les journalistes cherchent avant tout à délivrer une information de qualité",
    
    "ATTENTESJ_R": "Attentes relatives aux journalistes",
    "ATTENTESJ_R_1": "Sélectionner et hiérarchiser les informations les plus importantes",
    "ATTENTESJ_R_1": "Fournir des informations fiables et vérifiées",
    "ATTENTESJ_R_1": "Révéler les grands scandales politiques ou financiers",
    "ATTENTESJ_R_1": "Rester neutres en toutes circonstances",
    "ATTENTESJ_R_1": "Donner la parole à tout le monde y compris les minorités",
    "ATTENTESJ_R_1": "Permettre que tous les points de vue s'expriment librement",
    "ATTENTESJ_R_1": "Détecter les fausses nouvelles et lutter contre leur propagation",
    "ATTENTESJ_R_1": "Apporter des solutions aux différents problèmes",
    "ATTENTESJ_R_1": "Expliquer comment ils choisissent et fabriquent les informations",
    
    "DOUTE_R": "Doute sur la fiabilité des médias",
    "DEFIANCEJ_R": "Raison de douter de la fiabilité des médias",
    "DEFIANCEJ_R_1": "Les journalistes ne connaissent pas bien les sujets qu'ils traitent, ils manquent de rigueur",
    "DEFIANCEJ_R_2": "Les journalistes sont trop proches du pouvoir, des élites",
    "DEFIANCEJ_R_3": "Les journalistes travaillent dans de mauvaises conditions (manque de temps, manque de moyens...)",
    "DEFIANCEJ_R_4": "Les journalistes courent après les scoops, le sensationnalisme, pour plaire au public",
    "DEFIANCEJ_R_5": "Les journalistes vivent dans une bulle, ils ne connaissent pas la vraie vie des gens",
    "DEFIANCEJ_R_6": "Les journalistes sont soumis à la pression des propriétaires de journaux, de radio et de télévision",
    "DEFIANCEJ_R_7": "Les journalistes sont soumis à la pression des entreprises qui font de la publicité",
    "DEFIANCEJ_R_8": "Les journalistes sont soumis aux pressions politiques (gouvernement, partis politiques puissances étrangères)",
    "DEFIANCEJ_R_9": "Les journalistes défendent leur opinion au lieu de présenter des faits",
    "DEFIANCEJ_R_10": "(Ne pas citer : Aucune en particulier)",
    
    "CONNAISSALGOA_R": "Affirmations sur les réseaux sociaux",
    "CONNAISSALGOA_R_1": "Les réseaux sociaux choisissent de m'adresser certaines informations plutôt que d'autres",
    "CONNAISSALGOA_R_2": "Les informations que je reçois sur les réseaux sociaux dépendent des préférences (ou choix) des autres utilisateurs",
    "CONNAISSALGOA_R_3": "Les informations que je reçois sur les réseaux sociaux dépendent de ce que j'ai consulté ou aimé précédemment (de mon historique)",
    "CONNAISSALGOA_R_4": "Les informations négatives ou anxiogènes sont plus souvent diffusées que les autres par les réseaux sociaux",
    "CONNAISSALGOA_R_5": "Les réseaux sociaux diffusent à la fois des informations vraies et des informations fausses",
    "CONNAISSALGOA_R_6": "Il est possible de signaler une fausse information sur les réseaux sociaux",
    
    "CONNAISSALGOB_R": "Accord avec les affirmations que l'on pense exactes",
    "CONNAISSALGOB_R_1": "Les réseaux sociaux choisissent de m'adresser certaines informations plutôt que d'autres",
    "CONNAISSALGOB_R_2": "Les informations que je reçois sur les réseaux sociaux dépendent des préférences (ou choix) des autres utilisateurs",
    "CONNAISSALGOB_R_3": "Les informations que je reçois sur les réseaux sociaux dépendent de ce que j'ai consulté ou aimé précédemment (de mon historique)",
    "CONNAISSALGOB_R_4": "Les informations négatives ou anxiogènes sont plus souvent diffusées que les autres par les réseaux sociaux",
    "CONNAISSALGOB_R_5": "Les réseaux sociaux diffusent à la fois des informations vraies et des informations fausses",
    "CONNAISSALGOB_R_6": "Il est possible de signaler une fausse information sur les réseaux sociaux",

    "CRAINTESALGOIA_R": "Peurs vis-a-vis des réseaux sociaux",
    "SELECT_R": "Préférence vis-a-vis de la sélection des informations (Algorithmes vs rédaction)",
    "IA1_R": "Connaissance des utilisations de l'IA pour la création de contenu automatique",
    "OPTI_R": "Optimisme concernant l'avenir",
    "OPTI_R_1": "De votre situation personnelle",
    "OPTI_R_2": "De la France",
    "OPTI_R_3": "De l'Union Européenne",
    "OPTI_R_4": "Du climat, de l'environnement",
    "OPTI_R_5": "D'internet et des réseaux sociaux",
    "OPTI_R_6": "Des médias, des sources d'information",
    
    "ATTA_R": "Niveau d'attachement à...",
    "ATTA_R_1": "A votre pays",
    "ATTA_R_2": "A votre ville, votre village",
    "ATTA_R_3": "Au respect des règles démocratiques",
    "ATTA_R_4": "A la liberté d'expression",
    "ATTA_R_5": "A la diversité des opinions",
    "ATTA_R_6": "A la laïcité",
    "ATTA_R_7": "A la liberté de manifester",
    "ATTA_R_8": "Au droit de vote",
    
    "DEM_R": "Pensez-vous qu'en France, aujourd'hui,...",
    "DEM_R_1": "La démocratie fonctionne bien",
    "DEM_R_2": "Chacun peut exprimer librement son point de vue",
    "DEM_R_3": "Les médias sont libres",
    "DEM_R_4": "Tous les points de vue peuvent s'exprimer dans les médias",
    
    "CONFINST_R": "Niveau de confiance dans...",
    "CONFINST_R_1": "Les hommes et les femmes politiques",
    "CONFINST_R_2": "Les juges, les magistrats·es",
    "CONFINST_R_3": "Les policiers, policières",
    "CONFINST_R_4": "Les journalistes",
    "CONFINST_R_5": "Les enseignants·es",
    "CONFINST_R_6": "Le/la maire de votre commune",
    "CONFINST_R_7": "Les scientifiques",
    "CONFINST_R_8": "Les patrons, patronnes de médias",
    "CONFINST_R_9": "Les dirigeants·es des réseaux sociaux",
    "CONFINST_R_10": "Les créateurs, créatrices de contenus (Youtubeurs, influenceurs, streameurs, …)",

    "PERS_R": "Personnalité",
    "PERS_R_1": "J'aime être entouré·e de personnes, cultures, idées, styles de vie très différents",
    "PERS_R_2": "J'ai l'habitude de partager mes opinions",
    "PERS_R_3": "Je participe régulièrement à des discussions politiques avec mon entourage (mes amis, ma famille)",
    "PERS_R_4": "J'apprécie d'avoir plusieurs points de vue sur un même sujet",
    "PERS_R_5": "Je suis capable d'échanger avec des individus qui n'ont pas la même opinion que moi",
    "PERS_R_6": "Je pense que toutes les opinions doivent pouvoir s'exprimer",
    "INFLUENCE_R": "Vous demande-t-on souvent votre avis sur l'actualité?",
    
    "VA1_R": "Opinion sur des sujets clivants",
    "VA1_R_1": "Il faut aller plus loin dans l'égalité homme/femme",
    "VA1_R_2": "Aujourd'hui il y a trop d'immigrés en France",
    "VA1_R_3": "Il est important de reconnaitre la diversité des identités de genre (transgenres, non binaires)",
    "VA1_R_4": "Aujourd'hui l'avortement est trop facile d'accès",
    "VA1_R_5": "Il est normal que les couples homosexuels puissent avoir des enfants",
    "VA1_R_6": "Il faut changer nos modes de vie pour lutter contre le réchauffement climatique",
    "VA1_R_7": "Les Israéliens portent la plus grande part de responsabilité dans la poursuite du conflit israélo-palestinien",
    "VA1_R_8": "Les Français juifs ne sont pas des Français comme les autres",

    "NOU1_R": "Orientation politiue sur une échelle de 0 (gauche) à 10 (droite)",
    "PP3_R": "Vote au premier tour des présidentielles 2022",
    "QUIZZ1_R": "Israël a été récemment attaqué par...",
    "QUIZZ2_R": "Le premier ministre anglais s'appelle...",
    "QUIZZ3_R": "Le mandat actuel d'Emmanuel Macron se termine en...",
    "QUIZZ4_R": "En moyenne annuelle, en 2023 le taux d'inflation en France est de...",

    "QUESTION_OUVERTE": "Quelles sont les principales menaces pour la fiabilité de l'information?"
}







multiple_choice("MOTIV_R", 14)
multiple_choice("DEVICEXMOMENTS_R",7)
for i in range(7):
    multiple_choice(f"DEVICEXMOMENTS_R1_{i+1}", 8)
multiple_choice("FORMAT_R", 4)
multiple_choice("RSINFO1_LR_R",14)
multiple_choice("MOTIVRS_R", 12)
multiple_choice("SOURCES1AR_B1_R", 15)
multiple_choice("SOURCES1CR_B3_R", 17)
multiple_choice("SOURCES1DR_B4_R", 13)
multiple_choice("SOURCES1ER_B4_R", 9)
multiple_choice("SOURCES1FR_R", 14)
multiple_choice("TV2AR_R", 11)
multiple_choice("TV2BR_R", 14)
multiple_choice("TV2CR_R", 11)
multiple_choice("DYN2_R", 10)
multiple_choice("DYN3_R", 8)
multiple_choice("EVITE2_R", 9)
multiple_choice("INFOFIABLEDEF_R", 14)
multiple_choice("CRAINTESALGOIA_R", 8)

questionnaire

{'LEVEL_A': 'Records',
 'RECORDS': "Numéro d'enregistrement",
 'UUID': 'Identifiant du partcipant',
 'CIBLE': 'Cible',
 'SOURCE_BASE_R1': 'Source',
 'POIDS03': 'Poids',
 'RS1_R': 'Genre',
 'RS2C_RECODE_AG_R': 'Age',
 'RS3_R': 'Situation professionnelle',
 'RS4_R': 'Profession',
 'CSPIND_R': 'CSP',
 'RS5_R': "Niveau d'étude",
 'UDA9_R': 'Région en 9',
 'UDA5_R': 'Région en 5',
 'DPT_DOM': 'DOM',
 'CC_R': 'Taille agglo',
 'RS7_R': 'Nombre de personnes dans le foyer',
 'RS14_R': 'Revenu mensuel net du foyer',
 'ILLECTRO_R': "Dernière utilisation d'internet",
 'RS15_R': "Fréquence d'utilisation d'internet",
 'FREQ_INTERNET_R': "Fréquence d'utilisation d'internet (recode)",
 'RS16_R': 'Équipement média',
 'RS16_R_1': 'un poste de TV',
 'RS16_R_2': 'un poste de radio, une chaine hifi',
 'RS16_R_3': 'un autoradio',
 'RS16_R_4': 'un smartphone',
 'RS16_R_5': 'un ordinateur fixe',
 'RS16_R_6': 'un ordianteur portable',
 'RS16_R_7': 'une tablette tactile',
 'RS16_R_8': 'la google chromecast, le 

## Sauvegarde

In [79]:
import json

# Sauvegarde du dataframe au format csv
df_new.to_csv("Data/Etude sur les Français et l_Information/les-francais-et-l-information-propre.csv")

# Sauvegarde du dictionnaire au format json
with open("Data/Etude sur les Français et l_Information/questionnaire.json", "w") as f:
    json.dump(questionnaire, f, indent=4, ensure_ascii=False)

## Affichage

In [9]:
import json

# Chargement des données
df_new = pd.read_csv("Data/Etude sur les Français et l_Information/les-francais-et-l-information-propre.csv")

# Chargement des questions
with open("Data/Etude sur les Français et l_Information/questionnaire.json", "r") as f:
    questionnaire = json.load(f)

C:\Users\totog\AppData\Local\Temp\ipykernel_18736\3911797624.py:4: DtypeWarning: Columns (16,20,104,105,111,112,113,120,121,128,129,136,137,145,152,153,234,264,265,279,280,281,282,296,305,316,317,318,330,342,343,344) have mixed types. Specify dtype option on import or set low_memory=False.
  df_new = pd.read_csv("Data/Etude sur les Français et l_Information/les-francais-et-l-information-propre.csv")


In [17]:
for k in documentation.keys():
        liste_col = [col for col in df_new.columns if col.startswith(k)]
        
        if k in questionnaire.keys():
            print(questionnaire[k])
        else:
            print("ERROR")
        print(df_new[liste_col].iloc[300:310])
        print()

Cible
                     CIBLE
300  France métropolitaine
301  France métropolitaine
302  France métropolitaine
303  France métropolitaine
304  France métropolitaine
305  France métropolitaine
306  France métropolitaine
307  France métropolitaine
308  France métropolitaine
309  France métropolitaine

Source
     SOURCE_BASE_R1
300  CAWI Métropole
301  CAWI Métropole
302  CAWI Métropole
303  CAWI Métropole
304  CAWI Métropole
305  CAWI Métropole
306  CAWI Métropole
307  CAWI Métropole
308  CAWI Métropole
309  CAWI Métropole

Poids
               POIDS03
300  0,589587466769681
301  0,308866024033125
302  0,692116069804486
303   1,00769839429602
304  0,817504113746397
305  0,542297986592726
306  0,383125546026643
307   1,58123766926647
308  0,239135586253612
309  0,538028869234084

ERROR
     INTER
300    NaN
301    NaN
302    NaN
303    NaN
304    NaN
305    NaN
306    NaN
307    NaN
308    NaN
309    NaN

ERROR
     CONTACT
300      NaN
301      NaN
302      NaN
303      NaN
304      